In [27]:
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops
sess = tf.Session()

In [20]:
x_vals = np.array([1., 3., 5., 7., 9.])
x_data = tf.placeholder(tf.float32)
m_const = tf.constant(3.)
my_product = tf.multiply(x_data, m_const)
for x_val in x_vals:
    print(sess.run(my_product, feed_dict={x_data:x_val}))

3.0
9.0
15.0
21.0
27.0


In [21]:
# 3x5 array
my_array = np.array([[1., 3., 5., 7., 9.],
                     [-2., 0., 2., 4., 6.],
                     [-6., -3., 0., 3., 6.]])

# creates a second 3x5 array
x_vals = np.array([my_array, my_array + 1])

# Declare the tf placeholder
# If we do not know the shape beforehand, use tf.placeholder(tf.float32, shape=(3,None))
x_data = tf.placeholder(tf.float32, shape=(3, 5))

# Create three constants, size 5x1 and two scalers
m1 = tf.constant([[1.],[0.],[-1.],[2.],[4.]])
m2 = tf.constant([[2.]])
a1 = tf.constant([[10.]])

# Declare the operations
# matmul of 3x5 and 5x1, resulting in 3x1
prod1 = tf.matmul(x_data,m1)

# matmul of 3x1 and scaler, resulting in 3x1
prod2 = tf.matmul(prod1,m2)

# add of 3x1 and scaler
add1 = tf.add(prod2,a1)

for x_val in x_vals:
    print(sess.run(add1, feed_dict={x_data:x_val}))

[[102.]
 [ 66.]
 [ 58.]]
[[114.]
 [ 78.]
 [ 70.]]


In [31]:
# Example of feeding in an image, 1 image, size 4x4, depth 1)
x_shape = [1,4,4,1]
x_val = np.random.uniform(size=x_shape)
x_data = tf.placeholder(tf.float32,shape=x_shape)

In [23]:
my_filter = tf.constant(0.25, shape=[2,2,1,1])
my_strides = [1,2,2,1]

# image is 4x4, filter is 2x2, padding of zeros to keep it same size, stride of 2 makes the output 2x2
mov_avg_layer = tf.nn.conv2d(x_data,my_filter,my_strides,padding='SAME',name='Moving_Avg_Window')

In [24]:
# input our resulting 2x2 image, multiply it by another 2x2, and then add 1s to it, basically ax+b
def custom_layer(input_matrix):
    input_matrix_squeezed = tf.squeeze(input_matrix)
    # 2x2 constant
    A = tf.constant([[1.,2.],[-1.,3.]])
    B = tf.constant(1.,shape=[2,2])
    temp1 = tf.matmul(A,input_matrix_squeezed)
    temp = tf.add(temp1,B)
    return tf.sigmoid(temp)

In [27]:
# We created a custom layer above, but we need to put this layer on the graph with a named scope
# This makes it identifiable and collapsible/expandable on the computational graph
with tf.name_scope('Custom_Layer') as scope:
    custom_layer1 = custom_layer(mov_avg_layer)

In [30]:
print(sess.run(custom_layer1,feed_dict={x_data:x_val}))

[[0.87691295 0.9499861 ]
 [0.813603   0.90669864]]


In [2]:
# Implementing loss function, page 35
import matplotlib.pyplot as plt
import tensorflow as tf
sess = tf.Session()

In [3]:
x_vals = tf.linspace(-1., 1., 500)
target = tf.constant(0.)

In [5]:
# L2 norm loss, aka Euclidean loss function
# Square of the distance to the target, very curved near target
l2_y_vals = tf.square(target - x_vals)
l2_y_out = sess.run(l2_y_vals)

In [7]:
# L1 norm loss, aka absolute loss function, take absolute value of distance to target
# Better for outliers than L2, not as smooth near the target as L2 so may not converge as well
l1_y_vals = tf.abs(target - x_vals)
l1_y_out = sess.run(l1_y_vals)

In [ ]:
# Pseudo-Huber loss is continuous and smooth, attempts to take best part of L1 and L2 norms by being convex near target and less steep at extremes
# Form depends on extra parameter delta, dictates how steep it will be

In [20]:
# Classification loss functions used to evaluate loss when predicting categorical outcomes
# Redefine our predictions (x_vals) and target
x_vals = tf.linspace(-3., 5., 500)
target = tf.constant(1.)
targets = tf.fill([500,], 1.)


In [21]:
# Hinge loss mostly used with SVM, can be used in NN too. Computes loss between two target classes, 1 and -1
hinge_y_vals = tf.maximum(0., 1. - tf.multiply(target, x_vals))
hinge_y_out = sess.run(hinge_y_vals)

In [22]:
# Cross entropy loss for binary case, aka logistic loss, predicting two classes 0 and 1
# Measure distance from actual class (0 or 1) to the predicted value, usually a real number between 0 and 1
xentropy_y_vals = - tf.multiply(target, tf.log(x_vals)) - tf.multiply((1. -target), tf.log(1. - x_vals))
xentropy_y_out = sess.run(xentropy_y_vals)

In [23]:
# Sigmoid cross entropy loss, similar to cross entropy loss except we first transform x values by sigmoid function
xentropy_sigmoid_y_vals = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_vals,labels=targets)
xentropy_sigmoid_y_out = sess.run(xentropy_sigmoid_y_vals)

In [24]:
# Weighted cross entropy loss is weighted version of sigmoid cross entropy loss, provide a weight on the positive target

# Softmax cross-entropy loss operates on non-normalized outputs, measures loss when only one target category instead of multiple targets
# Transforms outputs into probability distribution via softmax function, computes loss function from true probability distribution

# Sparse softmax cross entropy loss is same as softmax cross entropy loss, but instead of target being probability distribution,
# Index of which category is true, pass in the index of which category is the true value

In [28]:
# Implementing back prop
ops.reset_default_graph()
sess = tf.Session()

In [29]:
x_vals = np.random.normal(1, 0.1, 100)
y_vals = np.repeat(10., 100)
x_data = tf.placeholder(shape=[1], dtype=tf.float32)
y_target = tf.placeholder(shape=[1], dtype=tf.float32)
A = tf.Variable(tf.random_normal(shape=[1]))

In [31]:
my_output = tf.multiply(x_data, A)

In [32]:
loss = tf.square(my_output - y_target)

In [33]:
init = tf.global_variables_initializer()
sess.run(init)

In [34]:
my_opt = tf.train.GradientDescentOptimizer(learning_rate=0.02)
train_step = my_opt.minimize(loss)

In [35]:
for i in range(100):
    rand_index = np.random.choice(100)
    rand_x = [x_vals[rand_index]]
    rand_y = [y_vals[rand_index]]
    sess.run(train_step, feed_dict={x_data: rand_x, y_target:rand_y})
    if (i+1)%25==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)))
        print('Loss = ' + str(sess.run(loss, feed_dict={x_data:rand_x, y_target: rand_y})))

Step #25 A = [6.593229]
Loss = [17.941595]
Step #50 A = [8.602763]
Loss = [0.4703996]
Step #75 A = [9.335105]
Loss = [1.6425915]
Step #100 A = [9.634427]
Loss = [0.00276154]


In [36]:
ops.reset_default_graph()
sess = tf.Session()

In [37]:
x_vals = np.concatenate((np.random.normal(-1, 1, 50), np.random.normal(3, 1, 50)))
y_vals = np.concatenate((np.repeat(0., 50), np.repeat(1., 50)))
x_data = tf.placeholder(shape=[1], dtype=tf.float32)
y_target = tf.placeholder(shape=[1], dtype=tf.float32)
A = tf.Variable(tf.random_normal(mean=10, shape=[1]))

In [38]:
my_output = tf.add(x_data, A)

In [39]:
my_output_expanded = tf.expand_dims(my_output, 0)
y_target_expanded = tf.expand_dims(y_target, 0)

In [40]:
init = tf.global_variables_initializer()
sess.run(init)

In [41]:
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=my_output_expanded,labels=y_target_expanded)

In [42]:
my_opt = tf.train.GradientDescentOptimizer(0.05)
train_step = my_opt.minimize(xentropy)

In [45]:
for i in range(1400):
    rand_index = np.random.choice(100)
    rand_x = [x_vals[rand_index]]
    rand_y = [y_vals[rand_index]]

    sess.run(train_step, feed_dict={x_data: rand_x, y_target:rand_y})
    if (i+1)%200==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)))
        print('Loss = ' + str(sess.run(xentropy, feed_dict={x_data: rand_x, y_target: rand_y})))

Step #200 A = [-0.86611]
Loss = [[0.24718349]]
Step #400 A = [-0.8413897]
Loss = [[0.08305367]]


Step #600 A = [-0.775075]
Loss = [[0.13139345]]


Step #800 A = [-0.92235]
Loss = [[0.65335244]]


Step #1000 A = [-0.8824818]
Loss = [[0.18552333]]
Step #1200 A = [-0.8412221]
Loss = [[0.06006026]]


Step #1400 A = [-0.99667853]
Loss = [[0.16939253]]


In [71]:
ops.reset_default_graph()
sess = tf.Session()

In [72]:
batch_size = 20
x_vals = np.random.normal(1, 0.1, 100)
y_vals = np.repeat(10., 100)
x_data = tf.placeholder(shape=[None, 1], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
A = tf.Variable(tf.random_normal(shape=[1,1]))

In [73]:
my_output = tf.matmul(x_data, A)

In [74]:
loss = tf.reduce_mean(tf.square(my_output - y_target))

In [77]:
my_opt = tf.train.GradientDescentOptimizer(0.02)
train_step = my_opt.minimize(loss)
init = tf.global_variables_initializer()
sess.run(init)

In [79]:
loss_batch = []
for i in range(100):
    rand_index = np.random.choice(100, size=batch_size)
    rand_x = np.transpose([x_vals[rand_index]])
    rand_y = np.transpose([y_vals[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    if (i+1)%100==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)))
        temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target : rand_y})
        print('Loss = ' + str(temp_loss))
        loss_batch.append(temp_loss)

Step #100 A = [[10.005499]]
Loss = 0.84329927
